* test the model on a df with an equal number of user types to test how good the model really is at classification

In [71]:
import pandas as pd
import numpy as np
from os.path import dirname, abspath

In [73]:
d = dirname(dirname(abspath("preprocessing.ipynb")))
df_wokring = pd.read_csv(d + '/data/Semi_processed_data.csv') #nrows = 1000)

df = df_wokring.sample(frac=0.03, replace=False, random_state=22) #3% smaple from df for 22794 observations

df.head()
df.shape

(22794, 12)

In [74]:
nans = lambda df: df[df.isnull().any(axis=1)]
nans

<function __main__.<lambda>(df)>

In [75]:
df.drop(['birth year','start station latitude','end station latitude',
        'start station longitude', 'end station longitude'], axis = 1, inplace = True)

In [76]:
print(df.shape)
df.head()

(22794, 7)


,tripduration,start station name,end station name,usertype,gender,Start Time,End Time
35922,224,Jersey & 6th St,Grove St PATH,Subscriber,1,32214.436,32438.436
493350,217,Grove St PATH,Marin Light Rail,Subscriber,1,73627.443,73845.094
558429,680,Riverview Park,Hilltop,Subscriber,1,76253.519,76934.417
285744,228,McGinley Square,Sip Ave,Subscriber,2,24314.744,24543.641
455471,387,Newport Pkwy,Harborside,Subscriber,2,58258.963,58646.882


In [77]:
df_Customers = df[df['usertype'] == 'Customer']
df_Subscribers = df[df['usertype'] == 'Subscriber']

In [78]:
print(df_Customers.shape)
print(df_Subscribers.shape)
df_Customers.head()

(1747, 7)
(21047, 7)


,tripduration,start station name,end station name,usertype,gender,Start Time,End Time
199183,301,Grove St PATH,Marin Light Rail,Customer,2,53853.371,54154.813
380568,273,McGinley Square,Sip Ave,Customer,0,56315.436,56589.436
562728,676,Hilltop,Grand St,Customer,0,8563.948,9240.721
586084,1391,Liberty Light Rail,Grand St,Customer,2,47530.919,48921.950
538032,818,Morris Canal,JC Medical Center,Customer,2,53443.451,54261.608


In [79]:
df_cust_sampled = df_Customers.sample(df_Customers.shape[0]//2)
df_subs_sampled = df_Subscribers.sample(df_Customers.shape[0]//2)
df_balanced = pd.concat([df_cust_sampled,df_subs_sampled], axis = 0)

In [80]:
print(df_balanced.shape)
df_balanced.head() 1746

(1746, 7)


,tripduration,start station name,end station name,usertype,gender,Start Time,End Time
404404,1209,Exchange Place,Liberty Light Rail,Customer,0,68472.132,69681.146
250825,1582,York St,Riverview Park,Customer,0,51327.256,52909.285
319443,1300,Newport Pkwy,Newport Pkwy,Customer,0,74054.543,75354.771
553427,2707,Washington St,Hamilton Park,Customer,0,57600.254,60307.968
686315,1135,Exchange Place,Lafayette Park,Customer,0,73523.190,74658.342


In [81]:
feature_names = ['start station name', 'end station name', 'gender',
                 'tripduration', 'Start Time','End Time']

label = 'usertype'
user_type = df_balanced[label]

In [82]:
df_balanced.drop(['usertype'], axis = 1, inplace = True)

In [83]:
nans = lambda df_balanced: df_balanced[df_balanced.isnull().any(axis=1)]
nans

<function __main__.<lambda>(df_balanced)>

In [84]:
print(df_balanced.shape)
df_balanced.head()

(1746, 6)


,tripduration,start station name,end station name,gender,Start Time,End Time
404404,1209,Exchange Place,Liberty Light Rail,0,68472.132,69681.146
250825,1582,York St,Riverview Park,0,51327.256,52909.285
319443,1300,Newport Pkwy,Newport Pkwy,0,74054.543,75354.771
553427,2707,Washington St,Hamilton Park,0,57600.254,60307.968
686315,1135,Exchange Place,Lafayette Park,0,73523.190,74658.342


In [85]:
user_type.head()

404404    Customer
250825    Customer
319443    Customer
553427    Customer
686315    Customer
Name: usertype, dtype: object

In [86]:
balance = user_type.value_counts(normalize = True)
balance

Customer      0.5
Subscriber    0.5
Name: usertype, dtype: float64

In [87]:
base = balance[0]
print("balance of the data / baseline accuracy = ",base)


balance of the data / baseline accuracy =  0.5


In [88]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import matplotlib
import statistics
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.compose import ColumnTransformer
import matplotlib
from matplotlib import pylab as plt
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

In [89]:
def ML_pipeline_kfold_GridSearchCV_log(X,y,random_state,n_folds):
    # create a test set
    X_other, X_test, y_other, y_test = train_test_split(X, y, test_size=0.2, random_state = random_state,stratify=y)
    # splitter for _other
    kf = StratifiedKFold(n_splits=n_folds,shuffle=True,random_state=random_state)
    # create the pipeline: preprocessor + supervised ML method
    
    feature_names = ['start station name', 'end station name', 'gender',
                     'tripduration', 'Start Time','End Time']

    cat_ftrs = ['start station name', 'end station name', 'gender']
    num_ftrs = ['tripduration', 'Start Time','End Time']
    
    cat_ftrs_i = [df_balanced.columns.get_loc(x) for x in cat_ftrs]
    num_ftrs_i = [df_balanced.columns.get_loc(x) for x in num_ftrs]
    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(sparse=False, handle_unknown = 'ignore'))])
    numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])


    # collect the transformers
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, num_ftrs_i),
            ('cat', categorical_transformer, cat_ftrs_i)]) 

    pipe = make_pipeline(preprocessor, LogisticRegression(penalty='l1', solver='saga',
                                                          max_iter=10000, multi_class = 'auto', random_state = 22))
    #pipe = make_pipeline(preprocess, LogisticRegression(penalty='l1', solver='saga', max_iter=10000))
    
    # the parameter(s) we want to tune
    param_grid = {'logisticregression__C': [0.1, 1.0, 10, 100]}
    # prepare gridsearch
    grid = GridSearchCV(pipe, param_grid=param_grid,scoring = make_scorer(accuracy_score),
                            cv=kf, return_train_score = True,iid=True)
    # do kfold CV on _other
    print("running")
    grid.fit(X_other, y_other)
    return grid, grid.score(X_test, y_test)




In [90]:
X = df_balanced.loc[:,df_balanced.columns != label].values
y = user_type.values

In [91]:
test_scores = []

for i in range(10):
    grid, test_score = ML_pipeline_kfold_GridSearchCV_log(X,y.ravel(),i*22, 5)
    print(grid.best_params_)
    print('best CV score:',grid.best_score_)
    print('test score:',test_score)
    test_scores.append(test_score)
    
print('test accuracy:',np.around(np.mean(test_scores),2),'+/-',np.around(np.std(test_scores),2))

mean_LR_a = np.mean(test_scores)
mean_LR_sd = np.std(test_scores)

running
{'logisticregression__C': 1.0}
best CV score: 0.8094555873925502
test score: 0.8542857142857143
running
{'logisticregression__C': 1.0}
best CV score: 0.8316618911174785
test score: 0.8114285714285714
running
{'logisticregression__C': 1.0}
best CV score: 0.8166189111747851
test score: 0.8457142857142858
running
{'logisticregression__C': 10}
best CV score: 0.8180515759312321
test score: 0.8171428571428572
running
{'logisticregression__C': 1.0}
best CV score: 0.826647564469914
test score: 0.8114285714285714
running
{'logisticregression__C': 1.0}
best CV score: 0.8173352435530086
test score: 0.8457142857142858
running
{'logisticregression__C': 1.0}
best CV score: 0.8187679083094556
test score: 0.8285714285714286
running
{'logisticregression__C': 100}
best CV score: 0.8244985673352435
test score: 0.8
running
{'logisticregression__C': 1.0}
best CV score: 0.8159025787965616
test score: 0.8228571428571428
running
{'logisticregression__C': 1.0}
best CV score: 0.8237822349570201
test sco

In [92]:
print("test accuracy:",mean_LR_a)
print("sd: ",mean_LR_sd) 

test accuracy: 0.8240000000000001
sd:  0.01807919087949302


In [93]:
LR_score = (mean_LR_a - base)/mean_LR_sd

print("LR_score =",LR_score) 

LR_score = 17.92115599418273


* **RF**

In [94]:
from sklearn.ensemble import RandomForestClassifier

In [100]:
def ML_pipeline_kfold_GridSearchCV_RF(X,y,random_state,n_folds):
    # create a test set
    X_other, X_test, y_other, y_test = train_test_split(X, y, test_size=0.2, random_state = random_state,stratify=y)
    # splitter for _other
    kf = StratifiedKFold(n_splits=n_folds,shuffle=True,random_state=random_state)
    # create the pipeline: preprocessor + supervised ML method
    
    feature_names = ['start station name', 'end station name', 'gender',
                     'tripduration', 'Start Time','End Time']

    cat_ftrs = ['start station name', 'end station name', 'gender']
    num_ftrs = ['tripduration', 'Start Time','End Time']
    
    cat_ftrs_i = [df_balanced.columns.get_loc(x) for x in cat_ftrs]
    num_ftrs_i = [df_balanced.columns.get_loc(x) for x in num_ftrs]

    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(sparse=False, handle_unknown = 'ignore'))])
    numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])


    # collect the transformers
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, num_ftrs_i),
            ('cat', categorical_transformer, cat_ftrs_i)]) 

    pipe = make_pipeline(preprocessor, RandomForestClassifier(random_state = 22))
    #pipe = make_pipeline(preprocess, LogisticRegression(penalty='l1', solver='saga', max_iter=10000))
    
    # the parameter(s) we want to tune
    param_grid = {'randomforestclassifier__min_samples_split': range(2,25,5),
                  'randomforestclassifier__max_depth': range(1,30,5)}
    # prepare gridsearch
    grid = GridSearchCV(pipe, param_grid=param_grid,scoring = make_scorer(accuracy_score),
                            cv=kf, return_train_score = True,iid=True)
    # do kfold CV on _other
    print("running")
    grid.fit(X_other, y_other)
    return grid, grid.score(X_test, y_test)


In [101]:
test_scores = []

for i in range(10):
    grid, test_score = ML_pipeline_kfold_GridSearchCV_RF(X,y.ravel(),i*22, 5)
    print(grid.best_params_)
    print('best CV score:',grid.best_score_)
    print('test score:',test_score)
    test_scores.append(test_score)
    
print('test accuracy:',np.around(np.mean(test_scores),2),'+/-',np.around(np.std(test_scores),2))

mean_RF_a = np.mean(test_scores)
mean_RF_sd = np.std(test_scores)



running
{'randomforestclassifier__max_depth': 26, 'randomforestclassifier__min_samples_split': 12}
best CV score: 0.8273638968481375
test score: 0.8485714285714285
running
{'randomforestclassifier__max_depth': 11, 'randomforestclassifier__min_samples_split': 7}
best CV score: 0.8431232091690545
test score: 0.8285714285714286
running
{'randomforestclassifier__max_depth': 21, 'randomforestclassifier__min_samples_split': 7}
best CV score: 0.8259312320916905
test score: 0.8371428571428572
running
{'randomforestclassifier__max_depth': 26, 'randomforestclassifier__min_samples_split': 12}
best CV score: 0.8388252148997135
test score: 0.8142857142857143
running
{'randomforestclassifier__max_depth': 26, 'randomforestclassifier__min_samples_split': 12}
best CV score: 0.8402578796561605
test score: 0.8171428571428572
running
{'randomforestclassifier__max_depth': 11, 'randomforestclassifier__min_samples_split': 2}
best CV score: 0.833810888252149
test score: 0.8457142857142858
running
{'randomfore

In [102]:
print("test accuracy:",mean_RF_a) 
print("sd: ",mean_RF_sd) 

test accuracy: 0.8262857142857142
sd:  0.013142857142857145


In [103]:
RF_score = (mean_RF_a - base)/mean_RF_sd

print("RF_score =",RF_score) 

RF_score = 24.826086956521728


* **saving RF**

In [98]:
def ML_pipeline_kfold_GridSearchCV_RF_3(X,y,random_state,n_folds):
    # create a test set
    X_other, X_test, y_other, y_test = train_test_split(X, y, test_size=0.2, random_state = random_state,stratify=y)
    # splitter for _other
    kf = StratifiedKFold(n_splits=n_folds,shuffle=True,random_state=random_state)
    # create the pipeline: preprocessor + supervised ML method
    
    feature_names = ['start station name', 'end station name', 'gender',
                     'tripduration', 'Start Time','End Time']

    cat_ftrs = ['start station name', 'end station name', 'gender']
    num_ftrs = ['tripduration', 'Start Time','End Time']
    
    cat_ftrs_i = [df_balanced.columns.get_loc(x) for x in cat_ftrs]
    num_ftrs_i = [df_balanced.columns.get_loc(x) for x in num_ftrs]

    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(sparse=False, handle_unknown = 'ignore'))])
    numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])


    # collect the transformers
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, num_ftrs_i),
            ('cat', categorical_transformer, cat_ftrs_i)]) 

    pipe = make_pipeline(preprocessor, RandomForestClassifier(random_state = 22))
    #pipe = make_pipeline(preprocess, LogisticRegression(penalty='l1', solver='saga', max_iter=10000))
    
    # the parameter(s) we want to tune
    param_grid = {'randomforestclassifier__min_samples_split': range(2,25,5),
                  'randomforestclassifier__max_depth': range(1,30,5)}
    # prepare gridsearch
    grid = GridSearchCV(pipe, param_grid=param_grid,scoring = make_scorer(accuracy_score),
                            cv=kf, return_train_score = True,iid=True)
    # do kfold CV on _other
    print("running")
    grid.fit(X_other, y_other)
    return grid, X_test, y_test



In [99]:
import pickle

best_estimators = []

for i in range(10):
    grid, X_test, y_test = ML_pipeline_kfold_GridSearchCV_RF_3(X,y,22*i,4)
    print(grid.best_score_)
    print(grid.score(X_test,y_test))
    print(grid.best_params_)
    best_estimators.append(grid.best_estimator_)
    
# save the output so I can use it later
file = open(d + '/results/RF Models_best_estimators_ballanced_data.save', 'wb')
pickle.dump((best_estimators),file)
file.close()



running
0.8223495702005731
0.86
{'randomforestclassifier__max_depth': 21, 'randomforestclassifier__min_samples_split': 22}
running
0.832378223495702
0.8028571428571428
{'randomforestclassifier__max_depth': 26, 'randomforestclassifier__min_samples_split': 2}
running
0.8273638968481375
0.8457142857142858
{'randomforestclassifier__max_depth': 21, 'randomforestclassifier__min_samples_split': 17}
running
0.833810888252149
0.8028571428571428
{'randomforestclassifier__max_depth': 16, 'randomforestclassifier__min_samples_split': 2}
running
0.835243553008596
0.8257142857142857
{'randomforestclassifier__max_depth': 26, 'randomforestclassifier__min_samples_split': 22}
running
0.8259312320916905
0.8171428571428572
{'randomforestclassifier__max_depth': 21, 'randomforestclassifier__min_samples_split': 17}
running
0.8302292263610315
0.82
{'randomforestclassifier__max_depth': 11, 'randomforestclassifier__min_samples_split': 2}
running
0.8316618911174785
0.7971428571428572
{'randomforestclassifier__max

* **SVM**

In [57]:
from sklearn.svm import SVC

In [58]:
def ML_pipeline_kfold_GridSearchCV_SVC(X,y,random_state,n_folds):
    # create a test set
    X_other, X_test, y_other, y_test = train_test_split(X, y, test_size=0.2, random_state = random_state,stratify=y)
    # splitter for _other
    kf = StratifiedKFold(n_splits=n_folds,shuffle=True,random_state=random_state)
    # create the pipeline: preprocessor + supervised ML method
    
    feature_names = ['start station name', 'end station name', 'gender',
                     'tripduration', 'Start Time','End Time']

    cat_ftrs = ['start station name', 'end station name', 'gender']
    num_ftrs = ['tripduration', 'Start Time','End Time']
    
    cat_ftrs_i = [df_balanced.columns.get_loc(x) for x in cat_ftrs]
    num_ftrs_i = [df_balanced.columns.get_loc(x) for x in num_ftrs]

    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(sparse=False, handle_unknown = 'ignore'))])
    numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])


    # collect the transformers
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, num_ftrs_i),
            ('cat', categorical_transformer, cat_ftrs_i)]) 

    pipe = make_pipeline(preprocessor, SVC(random_state = 22))
    #pipe = make_pipeline(preprocess, LogisticRegression(penalty='l1', solver='saga', max_iter=10000))
    
    # the parameter(s) we want to tune
    param_grid = {'svc__C': np.logspace(-3,4,num=8),'svc__gamma': np.logspace(-3,4,num=8)}
    # prepare gridsearch
    grid = GridSearchCV(pipe, param_grid=param_grid,scoring = make_scorer(accuracy_score),
                            cv=kf, return_train_score = True,iid=True)
    # do kfold CV on _other
    print("running")
    grid.fit(X_other, y_other)
    return grid, grid.score(X_test, y_test)

In [59]:
test_scores = []

for i in range(10):
    grid, test_score = ML_pipeline_kfold_GridSearchCV_SVC(X,y.ravel(),i*22, 5)
    print(grid.best_params_)
    print('best CV score:',grid.best_score_)
    print('test score:',test_score)
    test_scores.append(test_score)
    
print('test accuracy:',np.around(np.mean(test_scores),2),'+/-',np.around(np.std(test_scores),2))

mean_SVC_a = np.around(np.mean(test_scores),2)
mean_SVC_sd = np.around(np.std(test_scores),2)


running
{'svc__C': 1000.0, 'svc__gamma': 0.001}
best CV score: 0.8194842406876791
test score: 0.8457142857142858
running
{'svc__C': 10000.0, 'svc__gamma': 0.001}
best CV score: 0.8373925501432665
test score: 0.78
running
{'svc__C': 10000.0, 'svc__gamma': 0.001}
best CV score: 0.832378223495702
test score: 0.8342857142857143
running
{'svc__C': 100.0, 'svc__gamma': 0.01}
best CV score: 0.829512893982808
test score: 0.8171428571428572
running
{'svc__C': 10000.0, 'svc__gamma': 0.001}
best CV score: 0.828080229226361
test score: 0.8314285714285714
running
{'svc__C': 10000.0, 'svc__gamma': 0.001}
best CV score: 0.8237822349570201
test score: 0.8485714285714285
running
{'svc__C': 100.0, 'svc__gamma': 0.01}
best CV score: 0.8187679083094556
test score: 0.8542857142857143
running
{'svc__C': 10000.0, 'svc__gamma': 0.001}
best CV score: 0.8316618911174785
test score: 0.8142857142857143
running
{'svc__C': 1000.0, 'svc__gamma': 0.001}
best CV score: 0.832378223495702
test score: 0.8228571428571428


In [70]:
print(np.logspace(-3,4,num=8)) 

[1.e-03 1.e-02 1.e-01 1.e+00 1.e+01 1.e+02 1.e+03 1.e+04]


In [64]:
print("test accuracy:",mean_SVC_a)
print("sd: ",mean_SVC_sd)

test accuracy: 0.83
sd:  0.02


In [65]:
SVC_score = (mean_SVC_a - base)/mean_SVC_sd

print("SVC_score =",SVC_score)

SVC_score = 16.499999999999996
